In [6]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
# Data paths
train_dir = r"D:/Internships/Prodigy Infotech/DOGORCAT/train/train"
test_dir = r"D:/Internships/Prodigy Infotech/DOGORCAT/test1/test1"

# Image data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


# Mapping classes to labels
class_labels = {v:k for k, v in train_generator.class_indices.items()}


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [8]:
# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [9]:
# Train model
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=50,
                    validation_data=test_generator,
                    validation_steps=len(test_generator))

# Find best validation accuracy and its epoch
best_val_acc = max(history.history['val_accuracy'])
best_val_acc_epoch = history.history['val_accuracy'].index(best_val_acc) + 1



ValueError: Must provide at least one structure

In [ ]:
# Plot accuracies
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.scatter(best_val_acc_epoch, best_val_acc, color='red', marker='o', label=f'Best Val Acc: {best_val_acc:.2f}', s=100)
plt.xlabel('Epoch', fontsize=14, fontweight='bold', labelpad=10)
plt.ylabel('Accuracy', fontsize=14, fontweight='bold', labelpad=10)
plt.title('Training and Validation Accuracies', fontsize=16, fontweight='bold', pad=20)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Evaluate model
test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print("Test Accuracy:", test_accuracy)


In [ ]:
# Confusion matrix and classification report
predictions = model.predict(test_generator)
y_pred = (predictions > 0.5).astype("int32")
y_true = test_generator.classes
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['cat', 'dog']))

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\conta\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.